---

<p><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/50/Oracle_logo.svg/2560px-Oracle_logo.svg.png" width="200" align = "left"></p>



# **<h1 align ="right"><b> Detect Fashion objects in image</b></h1>**

---

In [ ]:
## tensorflow conda
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt


# **1. Model 1 - Yolov5**

---

In [ ]:
#!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [14]:
from PIL import Image
import torch

#model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')  # create model
model = torch.load('yolov5', 'custom', path='best.pt', source='local')

model.iou = 0.40  # NMS IoU threshold (0-1)
model.conf = 0.60  # confidence threshold (0-1)

#open image
img = Image.open(f'./images_clothing/img_3.jpg')

#apply model 1
results = model(img, size=640)

df_output = results.pandas().xyxy[0]
output_model_1 = df_output['name'].to_json(orient='records')
output_model_1

ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22432/501806733.py", line 5, in <cell line: 5>
    model = torch.load('yolov5', 'custom', path='best.pt', source='local')
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/torch/serialization.py", line 986, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/torch/serialization.py", line 435, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/torch/serialization.py", line 416, in __init__
    super().__init__(open(name, mode))
IsADirectoryError: [Errno 21] Is a directory: 'yolov5'
IsADirectoryError: [Er

## **3. Create boilerplate**

In [16]:
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os

  from ads.common.model_metadata import UseCaseType

  from ads.common.model_artifact import ModelArtifact

  from ads.common.model_export_util import prepare_generic_model



In [17]:
# !odsc conda init -b conda_environment_yolov5 -n frqap2zhtzbe -a resource_principal
# !odsc conda publish -s tensorflow28_p38_gpu_v1 --force

In [18]:
#path to artifacts and conda slug
path_to_artifacts = './artifacts_fashion_v1'
conda_env = 'oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1'   

#create default artifacts
artifact = prepare_generic_model(
    path_to_artifacts, 
    fn_artifact_files_included=False, 
    force_overwrite=True, 
    inference_conda_env=conda_env)

  artifact = prepare_generic_model(



loop1:   0%|          | 0/4 [00:00<?, ?it/s]

## **4. one script - score.py**

In [ ]:
## add the full yolov5 repo to the artifacst
## add the weights to the artifacst

In [ ]:
!cp ./yolov5 ./artifacts_fashion_v1

In [15]:
!cp best.pt ./artifacts_fashion_v1

In [19]:
%%writefile "{path_to_artifacts}/score.py"
from PIL import Image
import torch
import cv2
import numpy as np
import pandas as pd
import os
import io
import shutil
import sys
import glob
import ads
import urllib
from yolov5 import models, utils 
import base64
import uuid
import json

def load_model():
    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()

def predict(data, model=load_model()):

    print("Get image")
    #load base64 image from data
    #get the base64 images from the payload
    input_data = data['data']['input_image']
    
    #save image locally
    #input image folder
    path_input_image_locally = "/home/datascience/images" 
    
    #delete folder when exists
    if os.path.exists(path_input_image_locally):
        shutil.rmtree(path_input_image_locally)
    
    #make as new folder
    if not os.path.exists(path_input_image_locally):         
        os.makedirs(path_input_image_locally)
    
    print("Decode and save image")
    ##### decoding of image
    img_bytes_p = io.BytesIO(base64.b64decode(input_data.encode('utf-8')))
    input_image = Image.open(img_bytes_p).resize((224, 224))  
    
    #save image locally     
    input_image = input_image.save(path_input_image_locally + "/img_1.jpg")
    
    ####
    #### Start Model 1
    ####

    print("Start Model 1 - Yolov5")
    #load model
    model = torch.hub.load('yolov5', 'custom', path='best.pt', source='local')

    print("Model loaded")
    model.iou = 0.40  # NMS IoU threshold (0-1)
    model.conf = 0.60  # confidence threshold (0-1)

    #open image from locally
    img = Image.open(path_input_image_locally + "/img_1.jpg")

    print("apply model to image")
    #apply model 1
    results = model(img, size=640)

    df_output = results.pandas().xyxy[0]
    output_model_1 = df_output['name'].to_json(orient='records')
    print(output_model_1)

    
    return {'prediction': {'cloting_features': output_model_1}}

Overwriting ./artifacts_fashion_v1/score.py


## **Create payload image**

In [20]:
input_path="./img_6.jpg"

payload_string = str()
full_payload_string = str()
    
with open(input_path, "rb") as image2string:
    converted_string = base64.b64encode(image2string.read()).decode('ascii')
               
payload1 = json.dumps(converted_string)
json_payload1 = json.loads(payload1)
            
payload_json = {'data':{'input_image': json_payload1}}

###################################
# try function

predict(payload_json)

YOLOv5 🚀 v7.0-272-gde64179 Python-3.8.13 torch-2.1.0+cu121 CPU

Fusing layers... 


Get image
Decode and save image
Start Model 1 - Yolov5


Model summary: 476 layers, 87279442 parameters, 0 gradients
Adding AutoShape... 


apply model to image
["vest dress"]


{'prediction': {'cloting_features': '["vest dress"]'}}

In [21]:
%%writefile "{path_to_artifacts}/runtime.yaml"

# Model runtime environment
MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1
    INFERENCE_ENV_SLUG: tensorflow28_p38_gpu_v1
    INFERENCE_ENV_TYPE: published
    INFERENCE_PYTHON_VERSION: '3.8'
MODEL_PROVENANCE:
  PROJECT_OCID: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaangencdyaik5ssdqk4as2bhldxprh7vnqpk7yycsm7vymd344cgua
  TENANCY_OCID: ocid1.tenancy.oc1..aaaaaaaabu5fgingcjq3vc7djuwsdcutdxs4gsws6h4kfoldqpjuggxprgoa
  TRAINING_COMPARTMENT_OCID: ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: oci://conda_environment_yolov5@frqap2zhtzbe/conda_environments/gpu/TensorFlow 2.8 for GPU on Python 3.8/1.0/tensorflow28_p38_gpu_v1
    TRAINING_ENV_SLUG: tensorflow28_p38_gpu_v1
    TRAINING_ENV_TYPE: published
    TRAINING_PYTHON_VERSION: '3.8'
  TRAINING_REGION: eu-frankfurt-1
  TRAINING_RESOURCE_OCID: ocid1.datasciencenotebooksession.oc1.eu-frankfurt-1.amaaaaaangencdyacxmsz5ycch762wjc54udhibtl3m4nacuaf7shrvyoktq
  USER_OCID: ocid1.saml2idp.oc1..aaaaaaaar3ydw5hoiob7dfjzoom2dvbhqkkd5fat6m7upe72emlsxhsfrbfa/bob.peulen@oracle.com
  VM_IMAGE_INTERNAL_ID: NB1480-DCGPU131-VMP64-VMA1585-BI681

Overwriting ./artifacts_fashion_v1/runtime.yaml


## **Check artifacts**

In [22]:
#all should be passed
artifact.introspect()

['.ipynb_checkpoints', 'yolov5', 'score.py', 'runtime.yaml', 'test_json_output.json', 'best.pt']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


In [23]:
# Saving the model artifact to the model catalog. 
catalog_entry = artifact.save(display_name='clothing_detection_v1', description='clothing_detection_v1', timeout=600)
catalog_entry.id

  catalog_entry = artifact.save(display_name='clothing_detection_v1', description='clothing_detection_v1', timeout=600)



loop1:   0%|          | 0/5 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaangencdyaydz62h4jukgvduwl4af4jq4xugagc23vgael7fnbl5nq'

---

# **8. Deploy the ML Model and test real-time inference**

In [24]:
import requests
import oci
from oci.signer import Signer

In [25]:
uri = f"https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya4k5yytlulz2kvgvmjmo3qz3mmk4ed6zld5ksdi7upstq/predict"
print(uri)

https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya4k5yytlulz2kvgvmjmo3qz3mmk4ed6zld5ksdi7upstq/predict


In [26]:
config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])

import json

#POST request to the model
response = requests.post(uri, json=payload_json, auth=auth)
print(response)
xx = (json.loads(response.content))
print(xx)


<Response [200]>
{'prediction': {'cloting_features': '["vest dress"]'}}


---

## **Local UI for testing**

In [28]:
import requests
import oci
from oci.signer import Signer
import json

config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
        tenancy=config['tenancy'],
        user=config['user'],
        fingerprint=config['fingerprint'],
        private_key_file_location=config['key_file'],
        pass_phrase=config['pass_phrase'])



def full_function(input_image):
    
    input_path=input_image #is the path

    payload_string = str()
    full_payload_string = str()
    
    with open(input_path, "rb") as image2string:
        converted_string = base64.b64encode(image2string.read()).decode('ascii')

    payload1 = json.dumps(converted_string)
    json_payload1 = json.loads(payload1)

    payload_json = {'data':{'input_image': json_payload1}}

    uri = f"https://modeldeployment.eu-frankfurt-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdya4k5yytlulz2kvgvmjmo3qz3mmk4ed6zld5ksdi7upstq/predict"
    
    #POST request to the model
    response = requests.post(uri, json=payload_json, auth=auth)
    print(response)
    xx = (json.loads(response.content))    

    return xx

In [29]:
import gradio as gr

desc = "Clothing Detection"

with gr.Blocks() as demo: 
     
    input_image = gr.Image(label="source_city", type='filepath')

    xx = gr.Text(label='Detected clothes')


    submit_btn = gr.Button("Run Analysis")


gr.Interface(fn=full_function, inputs=input_image, outputs=xx, title=desc).launch(share=True, debug=True) #

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4a39dadd65c17de38c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


ERROR - Exception
Traceback (most recent call last):
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/gradio/blocks.py", line 2014, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22432/2999098039.py", line 15, in <cell line: 15>
    gr.Interface(fn=full_function, inputs=input_image, outputs=xx, title=desc).launch(share=True, debug=True) #
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/gradio/blocks.py", line 1926, in launch
    self.block_thread()
  File "/home/datascience/conda/tensorflow28_p38_gpu_v1/lib/python3.8/site-packages/gradio/blocks.py", line 2017, in block_thread
    self.server

Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

## **2. Model 2**

In [ ]:

from PIL import Image
from transformers import YolosFeatureExtractor, YolosForObjectDetection
import torch
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, ToPILImage



### Functions and labels
###

cats = ['shirt, blouse', 'top, t-shirt, sweatshirt', 'sweater', 'cardigan', 'jacket', 'vest', 'pants', 'shorts', 'skirt', 'coat', 'dress', 'jumpsuit', 'cape', 'glasses', 'hat', 'headband, head covering, hair accessory', 'tie', 'glove', 'watch', 'belt', 'leg warmer', 'tights, stockings', 'sock', 'shoe', 'bag, wallet', 'scarf', 'umbrella', 'hood', 'collar', 'lapel', 'epaulette', 'sleeve', 'pocket', 'neckline', 'buckle', 'zipper', 'applique', 'bead', 'bow', 'flower', 'fringe', 'ribbon', 'rivet', 'ruffle', 'sequin', 'tassel']

def fix_channels(t):
    """
    Some images may have 4 channels (transparent images) or just 1 channel (black and white images), in order to let the images have only 3 channels. I am going to remove the fourth channel in transparent images and stack the single channel in back and white images.
    :param t: Tensor-like image
    :return: Tensor-like image with three channels
    """
    if len(t.shape) == 2:
        return ToPILImage()(torch.stack([t for i in (0, 0, 0)]))
    if t.shape[0] == 4:
        return ToPILImage()(t[:3])
    if t.shape[0] == 1:
        return ToPILImage()(torch.stack([t[0] for i in (0, 0, 0)]))
    return ToPILImage()(t)
    
def idx_to_text(i):
    return cats[i]

# Random colors used for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        ax.text(xmin, ymin, idx_to_text(cl), fontsize=10,
                bbox=dict(facecolor=c, alpha=0.8))
    plt.axis('off')
    plt.show()
    plt.savefig("image.png")
    
    
def visualize_predictions(image, outputs, threshold=0.8):
    # keep only predictions with confidence >= threshold
    probas = outputs.logits.softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > threshold

    # convert predicted boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs.pred_boxes[0, keep].cpu(), image.size)

    # plot results
    plot_results(image, probas[keep], bboxes_scaled)

    
#############################

#define model
MODEL_NAME = "valentinafeve/yolos-fashionpedia"
feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-small')
model = YolosForObjectDetection.from_pretrained(MODEL_NAME)

#load iamge
image = Image.open(open('./img_1.jpg', "rb"))
image = fix_channels(ToTensor()(image))
image = image.resize((600, 800))

#apply model
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)